1. Selección del modelo


### Justificación:
- ResNet50 es conocido por su capacidad de manejar redes profundas sin problemas de degradación.
- VGG16 es simple y efectivo, con un buen equilibrio entre profundidad y rendimiento.

In [1]:
from torchvision import models

# Selección de modelos
model1 = models.resnet50(pretrained=True)
model2 = models.vgg16(pretrained=True)



c:\Users\Administrator\Documents\Projects\Covid\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\Documents\Projects\Covid\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Administrator/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 30.9MB/s]
c:\Users\Administrator\Documents\Projects\Covid\.venv\lib\site-packages\torchvision\models\_utils.py:223: User

2. Preprocesamiento de imágenes

In [ ]:
import cv2
import os
from torchvision import transforms
from PIL import Image

def preprocess_images(input_dir, output_dir, preprocess_type):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if preprocess_type == 'bilateral':
            processed_img = cv2.bilateralFilter(img, 9, 75, 75)
        elif preprocess_type == 'canny':
            processed_img = cv2.Canny(img, 100, 200)
        else:
            processed_img = img
        
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, processed_img)

# Directorios de entrada y salida
input_dir = 'dataset/2/Covid19-dataset/train'
output_dir_bilateral = 'dataset/output'
output_dir_canny = 'dataset/output'

# Preprocesamiento
preprocess_images(input_dir, output_dir_bilateral, 'bilateral')
preprocess_images(input_dir, output_dir_canny, 'canny')